## Hecho Por: Juan Felipe Camelo, Silvana Sandoval, Adrián Suárez

### Importación de las librerías a usar

In [1]:
import pyomo
import numpy as np
from scipy.optimize import linprog

# Ejercicio 1

# Ejercicio 2

Paso 1: Definir el problema teniendo en cuenta las variables de holgura. Esto se hace para convertir las desigualdades en igualdades. Para esto se transformará el problema: $$\text{max}(3x_1 + 2x_2)$$ en $$\text{max}(3x_1 + 2x_2+ 0s_1 + 0s_2 + 0s_3)$$ siendo $$s_1, s_2, s_3$$ las variables de holgura.

In [2]:
c = np.array([-3, -2, 0, 0, 0]) #Estás se añaden de manera negativa para poder añadirlas al simplex tableau después

Paso 2: Establecer la matriz de restricciones que se usará en las iteraciones. Esto se hace de esta manera, en una primera instancia, para entender con qué se está trabajando

In [3]:
tabla = {
    "Basic" : ['s1', 's2', 's3', 'Z'], #columna izquierda
    "x1": [2, 1, 1, -3],
    "x2": [1, 1, 0, -2],
    "s1": [1, 0, 0, 0],
    "s2": [0, 1, 0, 0],
    "s3": [0, 0, 1, 0],
    "b": [100, 80, 40, 0]
}

Paso 3: escribirla de tal manera que se pueda pasar por parámetro en una función sin problema alguno.

In [4]:
A = np.array([
    [2, 1, 1, 0, 0], 
    [1, 1, 0, 1, 0],
    [1, 0, 0, 0, 1]
], dtype=float)


Paso 4: escribir la columna b (la del lado derecho de la igualdad.)

In [5]:
b = np.array([100, 80, 40], dtype=float)

Paso 5: Construimos la tabla que tenga tanto la matriz A como el vector b. Además, le añadiremos una fila de costos para asegurarnos de poder identificar la columna pivote (es decir, la que reemplazará una de las variables slack).

In [6]:
tableau = np.hstack([A, b.reshape(-1, 1)]).astype(float) #Se convierte b en una matriz columna y se concatena con A
tableau = np.vstack([tableau, np.hstack([c, np.array([0])])]) #Se añade la matriz los costos hechos anteriormente

Paso 5: implementar el algoritmo simplex para resolver este problema.

In [7]:
def simplex(tableau, coordinates):

    num_rows, num_cols = tableau.shape

    x1, x2 = coordinates  # Coordenadas de las variables de decisión

    tableau[0, -1] = 100 - (2 * x1 + x2)  # Restricción 1: 2x1 + x2 <= 100
    tableau[1, -1] = 80 - (x1 + x2)       # Restricción 2: x1 + x2 <= 80
    tableau[2, -1] = 40 - x1              # Restricción 3: x1 <= 40

    # Iterar hasta que se cumpla la prueba de optimalidad (no haya negativos en la fila de Z)
    while np.any(tableau[-1, :-1] < 0):  # Si hay valores negativos, no es óptimo
        # 1. Identificar variable entrante: el valor más negativo en la fila de Z (función objetivo)
        pivot_col = np.argmin(tableau[-1, :-1])  # Columna con el valor más negativo
        
        # 2. Realizar la prueba de optimalidad: encontrar la variable saliente (la fila pivote)
        # Regla de la razón mínima: ratios de RHS / coeficientes de la columna pivote (sólo positivos)
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col],
                           out=np.full_like(tableau[:-1, -1], np.inf), 
                           where=tableau[:-1, pivot_col] > 0)
        pivot_row = np.argmin(ratios)  # Fila con el menor ratio positivo
        
        # 3. Realizar el pivoteo en torno al elemento pivote
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element  # Escalar la fila pivote para hacer el pivote = 1
        
        # Hacer ceros en las otras filas de la columna pivote
        for i in range(num_rows):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]
    
    # Cuando no haya valores negativos en la fila de Z, hemos alcanzado la optimalidad
    return tableau



Finalmente, nos encargaremos de calcular el tableua optimo e imprimirlo

In [8]:
coordenadas = np.array([0,0], dtype=float)
tableau_optimo = simplex(tableau=tableau, coordinates=coordenadas)
tableau_optimo = tableau_optimo.astype(int)
print("Optimal Tableau:\n", tableau_optimo)
print("Optimal Solution:", tableau_optimo[:, -1])

Optimal Tableau:
 [[  0   1  -1   2   0  60]
 [  0   0  -1   1   1  20]
 [  1   0   1  -1   0  20]
 [  0   0   1   1   0 180]]
Optimal Solution: [ 60  20  20 180]


Como se puede ver, los valores óptimos para $$x_1 = 40$$ y para $$x_2 = 100$$ 

Ahora para ver mejor el resultado se borrarán las filas que no son de las variables de holgura.

In [9]:
def contar_no_basicas(tabla):
    #primero, buscaremos el número de variables no báiscas dentro de la tabla de variables
    contador = 0
    for variable in tabla:
        if "x" in variable:
            contador +=1
    return contador

def eliminar_holgura(tabla, tableau):

    contador = contar_no_basicas(tabla)
    num_filas, num_columnas = tableau.shape

    #Ahora vamos a buscar las variables no basicas
    array = []
    for i in range (0, num_filas):
        element = tableau[i]

        for j in range (0, (contador)):
            x = element[j]
            if x == 1.:
                array.append(element.tolist())
    
    return array




In [10]:
array = eliminar_holgura(tabla=tabla, tableau=tableau)
for i in array:
    print(i)

[0.0, 1.0, -1.0, 2.0, 0.0, 60.0]
[1.0, 0.0, 1.0, -1.0, 0.0, 20.0]
